In [2]:
import gymnasium as gym
import tensorflow as tf
from tensorflow.keras.regularizers import l2
import numpy as np
import random

# Define the CNN model
def create_model(regularization_factor):
    #inputs = tf.keras.Input(shape=(4,))
    inputs = tf.keras.Input(shape=(8,)) #changed
    x = tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(regularization_factor))(inputs)
    x = tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(regularization_factor))(x)
    x = tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(regularization_factor))(x)
    #outputs = tf.keras.layers.Dense(2, activation="relu")(x)
    outputs = tf.keras.layers.Dense(4, activation="relu")(x) #changed
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="LunarLander")
    return model

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)



In [3]:
class Environment:
    def __init__(self, NUM_ENVS):
        self.num_envs = NUM_ENVS
        #self.envs = envs = gym.vector.make('CartPole-v1', num_envs=NUM_ENVS)#, render_mode='human') #changed
        self.envs = envs = gym.vector.make('LunarLander-v2', num_envs=NUM_ENVS)
        self.current_state, _ = self.envs.reset()

    def sample(self, model, epsilon):
        q_values = model(self.current_state) #get q values for current state
        action = np.argmax(q_values, axis=1) #get action that maximizes q-value #potential error?
        action = [random.choice(range(4)) if random.random() < epsilon else a for a in action] #choose epsilon greedy #changed
        new_observation, reward, terminated, truncated, info = self.envs.step(action)
        old_observation = self.current_state

        self.current_state = new_observation
        return (old_observation, action, reward, new_observation, terminated)


In [12]:
import pickle

class Buffer:
    def __init__(self, max_size, path=None):
        self.buffer = []
        self.max_size = max_size
        if path:
            self.load_from_file(path)

    def add_to_buffer(self, samples):
        #unpack the different environments
        old_obs_ = samples[0]
        actions_ = samples[1]
        rewards_ = samples[2]
        new_obs_ = samples[3]
        terminateds_ = samples[4]
        for o, a, r, no, t in zip(old_obs_, actions_, rewards_, new_obs_, terminateds_):
            self.buffer.append((o, a, r, no, t))
        if len(self.buffer) > self.max_size:
            self.buffer = self.buffer[-self.max_size:]

    def sample_minibatch(self, batch_size):
        r = random.sample(self.buffer, min(len(self.buffer), batch_size))
        return r
    
    def load_from_file(self, path):
        with open(path, 'rb') as f:
            lst = pickle.load(f)
        self.buffer += lst
        if len(self.buffer) > self.max_size:
            self.buffer = self.buffer[-self.max_size:]


In [13]:
b = Buffer(10000, 'kabuff.pkl')

In [14]:
len(b.buffer)

10000

In [5]:
TAU = 0.01
N = 50
K = 3
MINI_BATCH_SIZE = 64
CONVERGED = False
#MAX_STEPS = 2000#0
MAX_STEPS = 300
GAMMA = 0.99
regularization_factor = 0.001
epsilon = 1.0
minimum_epsilon = 0.01
epsilon_decay = 0.99


In [6]:

envs = Environment(7)

# Create the model
Q_theta = create_model(regularization_factor)
Q_target = tf.keras.models.clone_model(Q_theta)

kabuff = Buffer(100000)

STEPS = 0

while not CONVERGED and STEPS < MAX_STEPS:
    STEPS += 1
    
    Q_target.set_weights(Q_theta.get_weights())
    
    epsilon = max(minimum_epsilon, epsilon*epsilon_decay) 
    for n in range(N):
        
        kabuff.add_to_buffer(envs.sample(Q_theta, epsilon))
        for k in range(K):
            #sample s,a,r,s' minibatch from buffer
            minibatch = kabuff.sample_minibatch(MINI_BATCH_SIZE)

            new_states = [sample[3] for sample in minibatch]
            rewards = [sample[2] for sample in minibatch]
            actions = [sample[1] for sample in minibatch]
            old_states = [sample[0] for sample in minibatch]
            terminateds = [sample[4] for sample in minibatch]

            new_states = np.array(new_states)
            rewards = np.array(rewards)
            actions = np.array(actions)
            old_states = np.array(old_states)
            terminateds = np.array(terminateds)
            #everything seems fine

            Q_target_values = Q_target(new_states)
            Q_theta_values = Q_theta(new_states)
            #print("new states:", new_states)
            #print("Q_target_values", Q_target_values)
            #print("----------------------------------------------------------------------")
            #weirdly learns many zeros
            max_Q_target_values = []
            for target_action_vals, theta_action_vals in zip(Q_target_values, Q_theta_values):
                Q_theta_max_index = np.argmax(theta_action_vals)
                max_Q_target_value = target_action_vals[Q_theta_max_index]
                max_Q_target_values.append(max_Q_target_value)
            max_Q_target_values = np.array(max_Q_target_values)
            #max_Q_target_values = np.array([max(action_values) for action_values in Q_target_values])
            target_q_values = rewards + GAMMA * max_Q_target_values #this calculation works

            for i, tqv in enumerate(target_q_values):
                if terminateds[i]:
                    target_q_values[i] = rewards[i]

            with tf.GradientTape() as tape:
                predictions = Q_theta(old_states)
                #print(predictions)
                selected_q_values = tf.gather(predictions, actions, batch_dims=1)
                loss_value = tf.reduce_mean(tf.square(selected_q_values-target_q_values))
            gradients = tape.gradient(loss_value, Q_theta.trainable_variables)
            optimizer.apply_gradients(zip(gradients, Q_theta.trainable_variables))
    if STEPS % 10 == 0:
        print("last loss " + str(loss_value) + " in step " + str(STEPS))

last loss tf.Tensor(6.7166195, shape=(), dtype=float32) in step 10
last loss tf.Tensor(160.92351, shape=(), dtype=float32) in step 20
last loss tf.Tensor(38.680916, shape=(), dtype=float32) in step 30
last loss tf.Tensor(4.627407, shape=(), dtype=float32) in step 40
last loss tf.Tensor(162.02026, shape=(), dtype=float32) in step 50
last loss tf.Tensor(161.13817, shape=(), dtype=float32) in step 60


KeyboardInterrupt: 

In [6]:
#Q_theta.save('third_try')

INFO:tensorflow:Assets written to: third_try/assets


In [ ]:
Q_theta = keras.models.load_model("third_try")

In [7]:
test_env = gym.make('LunarLander-v2', render_mode='human')
#test_env = gym.make('CartPole-v1', render_mode='human')
obs, inf = test_env.reset()


In [9]:
for i in range(1000):
    qs = Q_theta(tf.expand_dims(obs, 0))
    act = np.argmax(qs)
    obs, _, terminated, _, _ = test_env.step(act)
    if(terminated):
        obs, _ = test_env.reset()

: 